# 8. Geography

Datashader contains a `geo` module which contains helper functions which should be familiar to the geospatial community. 

Some of the functions available include
* [Generate Terrain](#ds.geo---generate-terrain)
* [Hillshade](#ds.geo---hillshade-function)
* [Slope](#ds.geo---slope-function)
* [Aspect](#ds.geo---aspect-function)
* [Bump](#ds.geo---bump-function)
* [NDVI](#ds.geo---ndvi-function)
* [Mean](#ds.geo---mean-function)


In [ ]:
from datashader.transfer_functions import shade, stack
import numpy as np

## Generate Terrain Data

To demonstrate using these functions, let's generate some fake terrain...

In [ ]:
from datashader import Canvas
from datashader.geo import generate_terrain
from datashader.colors import Elevation

W = 1000
H = 750

canvas = Canvas(plot_width=W, plot_height=H, x_range=(-20e6, 20e6), y_range=(-20e6, 20e6))
terrain = generate_terrain(canvas)

The grayscale value above shows the elevation linearly in intensity (with the large black areas indicating low elevation), but it will look more like a landscape if we map the lowest values to colors representing water, and the highest to colors representing mountaintops:

In [ ]:
shade(terrain, cmap=Elevation, how='linear')

## Hillshade

[Hillshade](https://en.wikipedia.org/wiki/Terrain_cartography) is a technique used to visualize terrain as shaded relief, illuminating it with a hypothetical light source. The illumination value for each cell is determined by its orientation to the light source, which is based on slope and aspect.

In [ ]:
from datashader.geo import hillshade

illuminated = hillshade(terrain)

shade(illuminated, cmap=['gray', 'white'], alpha=255, how='linear')

You can combine hillshading with elevation colormapping to indicate terrain types:

In [ ]:
stack(shade(illuminated, cmap=['gray', 'white'], alpha=255, how='linear'),
      shade(terrain, cmap=Elevation, how='linear', alpha=128))

## Slope
[Slope](https://en.wikipedia.org/wiki/Slope) is the inclination of a surface. 
In geography, *slope* is amount of change in elevation of a terrain regarding its surroundings.

Datashader's slope function returns slope in degrees.  Below we highlight areas at risk for avalanche by looking at [slopes around 38 degrees](http://wenatcheeoutdoors.org/2016/04/07/avalanche-abcs-for-snowshoers/).

In [ ]:
from datashader.geo import slope

avalanche_slope_risk = slope(terrain)
avalanche_slope_risk.data = np.where(np.logical_and(avalanche_slope_risk.data > 25, 
                                     avalanche_slope_risk.data < 50),
                                     1, np.nan)

stack(
    shade(terrain, cmap=['black', 'white'], how='linear'),
    shade(illuminated, cmap=['black', 'white'], alpha=128, how='linear'),
    shade(avalanche_slope_risk, cmap='red', alpha=100), 
)

## Aspect

[Aspect](https://en.wikipedia.org/wiki/Aspect_(geography)) is the orientation of slope, measured clockwise in degrees from 0 to 360, where 0 is north-facing, 90 is east-facing, 180 is south-facing, and 270 is west-facing.

Below, we look to find slopes which face close to north.

In [ ]:
from datashader.geo import aspect

north_faces = aspect(terrain)
north_faces.data = np.where(np.logical_or(north_faces.data > 350 ,
                                          north_faces.data < 10), 1, np.nan)
stack(
    shade(terrain, cmap=['black', 'white'], how='linear'),
    shade(illuminated, cmap=['black', 'white'], alpha=128, how='linear'),
    shade(north_faces, cmap=['aqua'], alpha=50), 
)

## NDVI

The Normalized Difference Vegetation Index (NDVI) quantifies vegetation by measuring the difference between near-infrared (which vegetation strongly reflects) and red light (which vegetation absorbs).

For example, when you have negative values, it’s highly likely that it’s water. On the other hand, if you have a NDVI value close to +1, there’s a high possibility that it’s dense green leaves.
But when NDVI is close to zero, there isn’t green leaves and it could even be an urbanized area.

The output of *NDVI* ranges from [-1,+1], where `-1` means more "Red" radiation while `+1` means more "NIR" radiation.

Below, we simulate the red and near-infrared bands using `datashader.perlin` random noise with different seeds and frequencies.  Green areas should be those > 0, where higher NDVI values would indicate vegetation.

In [ ]:
from datashader.geo import ndvi
from datashader.geo import perlin

near_infrared_band = perlin(W, H, freq=(4, 3), seed=1)
red_band = perlin(W, H, freq=(32, 32), seed=2)
vegetation_index = ndvi(near_infrared_band, red_band)
shade(vegetation_index, cmap=['purple','black','green'], how='linear')

## Bump
Bump mapping is a cartographic technique often used to create the appearance of trees or other land features.

The `datashader.bump` will produce a bump aggregate that can then used to add detail to the terrain.  In this case, I will pretend the bumps are trees and shade them with green.

In [ ]:
from datashader.geo import bump
from functools import partial

def tree_heights(locations, min_val, max_val, height):
    out = np.zeros(len(locations))
    for i, (x, y) in enumerate(locations):
        val = terrain.data[y, x]
        if val > min_val and val < max_val:
            out[i] = height
        else:
            out[i] = 0
    return out

TREE_COUNT = 200000

trees = bump(W, H, count=TREE_COUNT // 3,
             height_func=partial(tree_heights, min_val=50, max_val=500, height=10))

trees += bump(W, H, count=TREE_COUNT,
             height_func=partial(tree_heights, min_val=500, max_val=2000, height=20))

trees += bump(W, H, count=TREE_COUNT // 3,
             height_func=partial(tree_heights, min_val=2000, max_val=3000, height=10))

tree_colorize = trees.copy()
tree_colorize.data[tree_colorize.data == 0] = np.nan

stack(shade(terrain + trees, cmap=['black', 'white'], how='linear'),
      shade(hillshade(terrain + trees), cmap=['black', 'white'], alpha=128, how='linear'),
      shade(tree_colorize, cmap='limegreen', how='linear'))

## Mean
The `datashader.mean` function will smooth a given aggregate by using a 3x3 mean convolution filter. Optional parameters include `passes`, which is used to run the mean filter multiple times, and also `excludes` which are values that will not be modified by the mean filter.

Just for fun, let's add a coastal vignette to give out terrain scene a bit more character. Notice the water below now has a nice coastal gradient which adds some realism to our scene.

In [ ]:
from datashader.geo import mean

LAND_CONSTANT = 50.

water = terrain.copy()
water.data = np.where(water.data > 0, LAND_CONSTANT, 0)
water = mean(water, passes=10, excludes=[LAND_CONSTANT])
water.data[water.data == LAND_CONSTANT] = np.nan

stack(
    shade(terrain, cmap=['black', 'white'], how='linear'),
    shade(water, cmap=['aqua','white']),
    shade(hillshade(terrain), cmap=['black', 'white'], alpha=128, how='linear'),
)

## Conclusion

We've now seen a bunch of datashader's `geo` helper functions.

Let's make our final archipelago scene by stacking `terrain`, `water`, `hillshade`, and `tree_highlights` together into one output image: 

In [ ]:
stack(shade(terrain + trees, cmap=Elevation, how='linear'),
      shade(water, cmap=['aqua','white']),
      shade(hillshade(terrain + trees), cmap=['black', 'white'], alpha=128, how='linear'),
      shade(tree_colorize, cmap='limegreen', how='linear'))

### References
- Burrough, P. A., and McDonell, R. A., 1998. Principles of Geographical Information Systems (Oxford University Press, New York), pp 406
- Making Maps with Noise Functions: https://www.redblobgames.com/maps/terrain-from-noise/
- How Aspect Works: http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/how-aspect-works.htm#ESRI_SECTION1_4198691F8852475A9F4BC71246579FAA